In [1]:
import os
import numpy as np
import palpy
import lsst.sims.skybrightness as sb
import lsst.sims.ocs.configuration
from lsst.sims.utils import altAzPaFromRaDec, ObservationMetaData

In [2]:
field_dict = {}
with open("tessellationFields") as fh:
    field_id = 1
    for line in fh:
        line = line.strip()
        # Skip blank line
        if not line:
            continue
        # Skip comment lines
        if line.startswith("#"):
            continue
        values = (float(x) for x in line.split())
        field = lsst.sims.ocs.configuration.Field(field_id, 3.5, *values)
        field_dict[field_id] = field
        field_id += 1

In [3]:
# Default is LSST observatory and preciseAltAz off but give back LSST filter magnitudes 
sm = sb.SkyModel(mags=True)

In [4]:
mjd = 59581.041666666664
ra = np.array([field.ra for _, field in field_dict.iteritems()])
dec = np.array([field.dec for _, field in field_dict.iteritems()])
obs = ObservationMetaData(mjd=mjd)

alt, az, pa = altAzPaFromRaDec(ra, dec, obs)

/home/mareuter/lsst/Linux64/sims_utils/sims_2.2.2+1/python/lsst/sims/utils/ModifiedJulianDate.py:102: UserWarning: UTC 5.958104e+04 is outside of IERS table for UT1-UTC.
Returning UT1 = UTC for lack of a better idea
  + "Returning UT1 = UTC for lack of a better idea")
/home/mareuter/lsst/Linux64/sims_utils/sims_2.2.2+1/python/lsst/sims/utils/ModifiedJulianDate.py:81: UserWarning: UTC 5.958104e+04 is outside of IERS table for UT1-UTC.
Returning UT1 = UTC for lack of a better idea
  + "Returning UT1 = UTC for lack of a better idea")


In [5]:
alt

array([ 30.34537627,  28.62848452,  30.67382062, ..., -29.23353408,
       -27.18671828, -28.90499397])

In [6]:
alt_mask = np.where(alt > 20.0)
print(alt_mask)
up_ra = ra[alt_mask]
up_dec = dec[alt_mask]
up_alt = alt[alt_mask]
up_az = az[alt_mask]
print(up_alt.size)

(array([   0,    1,    2, ..., 4304, 4312, 4322]),)
1741


In [7]:
sm.setRaDecMjd(up_ra, up_dec, mjd, degrees=True)

/home/mareuter/git/lsst/sims_skybrightness/python/lsst/sims/skybrightness/skyModel.py:63: RuntimeWarning: invalid value encountered in arccos
  az = np.arccos((sindec-np.sin(alt)*sinlat)/(np.cos(alt)*coslat))
/home/mareuter/git/lsst/sims_skybrightness/python/lsst/sims/skybrightness/skyModel.py:438: RuntimeWarning: invalid value encountered in greater
  over = np.where(self.azRelMoon > np.pi)
/home/mareuter/lsst/Linux64/healpy/1.8.1+17/lib/python/healpy-1.8.1-py2.7-linux-x86_64.egg/healpy/pixelfunc.py:1045: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  if phi == None:


In [8]:
# Here are the access patterns to get at the intermediate information calculated by the SkyModel.at
attrs = sm.getComputedVals()

# These parameters all have the size of the input RA, DEC list.
# NOTE: All units are radians unless specified.

# Field airmass
print(attrs["airmass"])
# Field Altitudes
print(attrs["alts"])
# Field Azimuths
print(attrs["azs"])
# Field ecliptic latitude
print(attrs["eclipLat"])
# Field ecliptic longitude
print(attrs["eclipLon"])

[ 1.98534993  2.09321225  1.966972   ...,  2.865936    2.88272021
  2.90875171]
[ 0.52786436  0.49807424  0.53332067 ...,  0.35642499  0.35425804
  0.35095003]
[        nan  3.10690223  3.19038731 ...,  0.12205063  0.06935841
  0.01680829]
[-1.16170352 -1.1596319  -1.13548092 ...,  0.33044421  0.35015253
  0.36932067]
[ 4.71238898  4.60604296  4.79386937 ...,  1.07041552  1.02230529
  0.97362889]


In [9]:
# Here are the access patterns to get at the intermediate information calculated by the SkyModel.

# These parameters are all single valued.
# NOTE: All units are radians unless specified.

print(attrs["sunAlt"])
print(attrs["sunAz"])
print(attrs["sunRA"])
print(attrs["sunDec"])
print(attrs["sunEclipLon"])
print(attrs["moonPhase"])
print(attrs["moonAlt"])
print(attrs["moonAz"])
print(attrs["moonRA"])
print(attrs["moonDec"])


-13:55:22.3
232:28:08.1
18:50:23.93
-22:55:49.8
281:17:36.6
0.86925560236
-19:49:12.5
222:36:27.7
18:02:45.26
-25:18:03.7


In [10]:
# Calculate the Field, Moon separation in radians.
moonSep = palpy.dsepVector(up_ra, up_dec, np.full_like(up_ra, attrs["moonRA"]), np.full_like(up_dec, attrs["moonDec"]))
print(moonSep)

[ 1.18402223  1.05028446  1.40676645 ...,  1.70124505  1.99301376
  1.96157418]


In [11]:
mags = sm.returnMags()

In [12]:
# For each RA,DEC you get a ugrizy map
i_filt = 3
bad_count = 0
for ra, dec, alt, az, fmag in zip(up_ra, up_dec, up_alt, up_az, mags):
    i_mag = fmag[i_filt]
    if i_mag == np.Inf:
        bad_count += 1
    print(ra, dec, alt, az, i_mag)
print("Number of bad fields: {} out of {}".format(bad_count, up_ra.size))

(0.0, -90.0, 30.345376266467646, 180.09527155034436, 19.367914101768378)
(180.0, -87.568555000000003, 28.628484517578688, 178.11934388920477, 19.11852208987024)
(324.000429, -87.568550000000002, 30.673820621229673, 182.89104307183246, 19.186312267079892)
(35.999571000000003, -87.568550000000002, 32.740622992961619, 180.57330183382987, 19.287543471942644)
(252.001105, -87.568546999999995, 28.16512868052487, 181.32440563203664, 19.072419490045917)
(107.998895, -87.568546999999995, 31.443857881276394, 177.56835942640558, 19.250129713695259)
(215.99982199999999, -85.272913000000003, 25.685022525096972, 179.22856568825128, 18.940200987000789)
(144.00017800000001, -85.272913000000003, 29.539288072047285, 174.72070079943745, 19.183179436058385)
(0.0, -85.272891999999999, 33.592759707175048, 184.14238846838666, 19.303285403112991)
(288.00050800000002, -85.272852, 28.077125181878454, 184.84562592146685, 19.04313746371141)
(71.999492000000004, -85.272852, 34.548933141812903, 177.53909462061873, 